In [63]:
import csv
import pandas as pd
import re
import string
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vmakh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
# define function to preprocess the messages
def preprocess_message(message):
    # remove semicolons and quotes
    message = re.sub('[;"]+', '', message)

    # convert to lowercase
    message = message.lower()

    # remove punctuation
    message = message.translate(str.maketrans('', '', string.punctuation))

    # remove stopwords
    stop_words = set(stopwords.words('russian')) 
    message_tokens = message.split()
    filtered_tokens = [token for token in message_tokens if token not in stop_words]
    message = ' '.join(filtered_tokens)

    return message

# read the csv file
df = pd.read_csv('negative.csv', sep=';', header=None, usecols=[3, 4], names=['message', 'label'])

In [65]:
df['message'] = df['message'].apply(preprocess_message)
df.head()

,message,label
0,работе полный пиддес каждое закрытие месяца св...,-1
1,коллеги сидят рубятся urban terror изза долбан...,-1
2,elina4post говорят обещаного года ждут,-1
3,желаю хорошего полёта удачной посадкия буду оч...,-1
4,обновил какимто лешим surf работает простоплеер,-1


In [66]:
tweets=df['message'].to_list()
tweets[:10]

['работе полный пиддес каждое закрытие месяца свихнусь d',
 'коллеги сидят рубятся urban terror изза долбанной винды могу',
 'elina4post говорят обещаного года ждут',
 'желаю хорошего полёта удачной посадкия буду очень сильно скучать httptcojclnzvnv3s',
 'обновил какимто лешим surf работает простоплеер',
 'котёнка вчера носик разбила плакала расстраивалась',
 'juliamayko onika55 andpossum зашли затихарился прямо физически страдаю долго молчит',
 'вообще болею выздоравливаю',
 'микрофраза учимся срать кирпичами режиме ampquotнонстопampquot niwoqisipapy',
 'хочу тобой помириться сука гордая сделаю']

In [68]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Обучение модели word2vec на списке твитов
tweets_processed = [simple_preprocess(tweet) for tweet in tweets]
model = Word2Vec(tweets_processed, min_count=1, vector_size=100)

# Получение векторного представления каждого твита
tweet_vectors = []
for tweet in tweets_processed:
    # Проверяем, не является ли твит пустым
    if tweet:
        tweet_vec = np.mean([model.wv[word] for word in tweet if word in model.wv], axis=0)
        tweet_vectors.append(tweet_vec)
    else:
        # Если твит пустой, добавляем вектор из нулей
        tweet_vectors.append(np.zeros(model.vector_size))

def find_similar_tweets(query, num_results=5):
    query_words = [word for word in simple_preprocess(query) if word in model.wv]

    # Проверяем, не является ли запрос пустым
    if not query_words:
        return []

    query_vec = np.mean([model.wv[word] for word in query_words], axis=0)
    similarities = cosine_similarity([query_vec], tweet_vectors)[0]
    similar_indices = similarities.argsort()[::-1][:num_results]
    return [tweets[i] for i in similar_indices]


In [69]:
query = "работаю над проектом"
results = find_similar_tweets(query, num_results=5)
print(results)




['думаю джастин селена снова вместе', '2 месяца английского школе учительница болеет awful hateit', 'геологию снова сделал факфакфак щит', 'пипецкогда закончится эта неделя снова проспала', 'школе номальная ёлка хрень']
